In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
    # for filename in filenames:
    #     print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input
/kaggle/input/my_temp
/kaggle/input/my_temp/pytorch
/kaggle/input/my_temp/pytorch/default
/kaggle/input/my_temp/pytorch/default/1
/kaggle/input/my-temp
/kaggle/input/pm-71595720-at-11-25-2024-13-31-01


In [2]:
from ultralytics import YOLO
from pytubefix import YouTube  # pytube-fix 사용
import cv2

youtube_url = 'https://www.youtube.com/watch?v=IaLPsDV9AbQ'

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
def download_youtube_video(url, output_path="/kaggle/working/output"):
    yt = YouTube(url)
    video_stream = yt.streams.filter(file_extension="mp4", res="720p").first()
    if not video_stream:
        video_stream = yt.streams.get_highest_resolution()
    os.makedirs(output_path, exist_ok=True)
    video_path = os.path.join(output_path, yt.title + ".mp4")
    video_stream.download(output_path)
    return video_path


if not os.path.exists('/kaggle/input/my-temp/sample_video.mp4'):
    downloaded_video = download_youtube_video(youtube_url)

In [4]:
# !rm /kaggle/working/output/result_sample_video.mp4

In [5]:
from typing import Tuple
from datetime import datetime

# Reference : https://gist.github.com/IdeaKing/11cf5e146d23c5bb219ba3508cca89ec
def resize_with_pad(image: np.array, 
                    new_shape: Tuple[int, int], 
                    padding_color: Tuple[int] = (0, 0, 0)) -> np.array:
    """Maintains aspect ratio and resizes with padding.
    Params:
        image: Image to be resized.
        new_shape: Expected (width, height) of new image.
        padding_color: Tuple in BGR of padding color
    Returns:
        image: Resized image with padding
    """
    original_shape = (image.shape[1], image.shape[0])
    ratio = float(max(new_shape))/max(original_shape)
    new_size = tuple([int(x*ratio) for x in original_shape])
    image = cv2.resize(image, new_size)
    delta_w = new_shape[0] - new_size[0]
    delta_h = new_shape[1] - new_size[1]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=padding_color)
    return image


def process_video_with_yolo(video_path, model_path="/kaggle/input/my_temp/pytorch/default/1/best.pt", output_path="/kaggle/working/output"):
    model = YOLO(model_path)  # YOLO 모델 로드
    cap = cv2.VideoCapture(video_path)
    os.makedirs(output_path, exist_ok=True)

    # 비디오 저장 설정
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    print(f'{width} {height} {fps}')
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 코덱 설정
    out_path = os.path.join(output_path, "result_" + os.path.basename(video_path))
    out = cv2.VideoWriter(out_path, fourcc, fps, (640, 640))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 프레임 리사이즈 (640x640)
        start_time = datetime.now()
        # resized_frame = cv2.resize(frame, (640, 640))
        resized_frame = resize_with_pad(frame, new_shape=[640, 640],padding_color= [0, 0, 0])
        end_time = datetime.now()
        elapsed_time = end_time - start_time
        print(f'[Info] elapsed_time 4 resize={elapsed_time.total_seconds()}s')
        
        # YOLO 추론
        results = model(resized_frame)
        
        # 결과 시각화
        annotated_frame = results[0].plot()  # 탐지 결과를 프레임에 그리기
        out.write(annotated_frame)  # MP4에 저장

    cap.release()
    out.release()
    print(f"MP4 저장 완료: {out_path}")
    return out_path


if not os.path.exists('/kaggle/input/my-temp/sample_video.mp4'):
    result_video = process_video_with_yolo(video_path='/kaggle/working/output/꿀벌 날개가 이상해요... 진드기 피해가 심각하네요.  꿀벌아 미안해.mp4')    
else:
    result_video = process_video_with_yolo(video_path='/kaggle/input/my-temp/sample_video.mp4')

1280 720 30
[Info] elapsed_time 4 resize=0.006873s

0: 640x640 16 정상s, 7.6ms
Speed: 7.1ms preprocess, 7.6ms inference, 244.6ms postprocess per image at shape (1, 3, 640, 640)


100%|██████████| 22.2M/22.2M [00:00<00:00, 45.1MB/s]

[Info] elapsed_time 4 resize=0.001833s

0: 640x640 16 정상s, 6.7ms
Speed: 4.3ms preprocess, 6.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
[Info] elapsed_time 4 resize=0.000691s

0: 640x640 19 정상s, 6.3ms
Speed: 3.2ms preprocess, 6.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
[Info] elapsed_time 4 resize=0.000814s



0: 640x640 19 정상s, 7.7ms
Speed: 3.2ms preprocess, 7.7ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
[Info] elapsed_time 4 resize=0.000659s

0: 640x640 16 정상s, 6.4ms
Speed: 2.0ms preprocess, 6.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
[Info] elapsed_time 4 resize=0.000686s

0: 640x640 17 정상s, 7.0ms
Speed: 2.3ms preprocess, 7.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
[Info] elapsed_time 4 resize=0.000665s

0: 640x640 16 정상s, 6.2ms
Speed: 2.0ms preprocess, 6.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
[Info] elapsed_time 4 resize=0.000741s

0: 640x640 16 정상s, 6.8ms
Speed: 2.7ms preprocess, 6.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
[Info] elapsed_time 4 resize=0.000608s

0: 640x640 15 정상s, 6.5ms
Speed: 1.8ms preprocess, 6.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
[Info] elapsed_time 4 resize=0.000652s

0: 640x640 15 정상s, 6.7ms
Speed: 1.